In [1]:
import sys
sys.path.append('..')
from Comms_System import Comms_System, SNR_plot, network_sender_receiver
from filters import butter_lowpass
import numpy as np
import matplotlib.pyplot as plt
import torch
#from NetworkPytorch import joint_train_loop
from scipy import signal
import scipy
import torchaudio
from scipy.stats import norm
from commpy import rcosfilter
from DE_Pytorch_Joint import DE
from DE_Pytorch_Joint_Minibatch import DE as DEM
import random

In [2]:
def get_data():

    upsampled = CS.upsample()
    X_tx = torch.Tensor(upsampled).view(1, 1, -1)
    classes = np.array(symbol_set)
    y = symbol_seq
    class_idx = {v: i for i, v in enumerate(classes)}
    y_idx = np.array([class_idx[v] for v in y])
    y = torch.Tensor(y_idx)
    
    return X_tx, y

In [3]:
def make_nets():
    NN_tx = torch.nn.Sequential(torch.nn.Conv1d(1, 1, 64, padding=63)) # padding=len(CS.h) - 1
    NN_rx = torch.nn.Sequential(torch.nn.Conv1d(1, 1, 64), torch.nn.Conv1d(1, 4, 8, stride=8))
    for param1, param2 in zip(NN_tx.parameters(), NN_rx.parameters()):
        param1.requires_grad = False
        param2.requires_grad = False
    return (NN_tx, NN_rx)

In [4]:
symbol_set = [3, 1, -1, -3] # all symbols that we use
num_symbols = 100000
symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
m = 8
CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m, beta=0.35)
classes = np.array(symbol_set)


Xtrain, ytrain = get_data()

D = DE(objective_function=torch.nn.CrossEntropyLoss(), pop_size=20, pop_fun=make_nets, 
      X=Xtrain, y=ytrain, F=0.6, cr=0.85, use_cuda=False, SNRdb=6.5)

In [ ]:
D.evolution(num_epochs=1000, verbose=True, print_epoch=100)

In [ ]:
lf_tx = list(D.best_agent[0].parameters())[0].detach()[0][0]
lf_rx = list(D.best_agent[1].parameters())[0].detach()[0][0]
plt.plot(lf_tx)
plt.show()
plt.plot(lf_rx)
plt.show()

In [ ]:
# Evaluate

accs = np.zeros(10)
accs_euclid = np.zeros(10)
NN_tx = D.best_agent[0]
NN_rx = D.best_agent[1]

for i in range(10):

    SNR = 6.5
    joint_decisions = CS.transmission(SNRdb=SNR, mode='joint', joint_cutoff=0.675, model=[NN_tx, NN_rx], lowpass='butter')
    euclid_decisions = CS.transmission(SNRdb=SNR, mode='euclidean')
    acc = 1 - CS.evaluate(joint_decisions)[1]
    acc_euclid = 1 - CS.evaluate(euclid_decisions)[1]

    #print('Accuracy:', acc)
    #print('Accuracy:', 1 - CS.evaluate(euclid_decisions)[1])
    accs[i] = acc
    accs_euclid[i] = acc_euclid
    
print(np.mean(accs))
print(np.mean(accs_euclid))